In [35]:
file = open("Airbnb_Texas_Rentals.csv",encoding="utf8")
lines = file.readlines()[1:10]
for line in lines:
    cols = line.replace('"', '').replace('\t', ' ').split(',')
    record = "\t".join(cols[1:])
    with open('doc\doc_{}.tsv'.format(cols[0]), 'w',encoding="utf8") as file:
            file.write(record)


In [36]:
from os import listdir
from os.path import isfile, join
docpaths = [f for f in listdir("doc") if isfile(join("doc", f))]


In [37]:
def save_dict_to_file(dic, file):
    f = open('{}.txt'.format(file), 'w')
    f.write(str(dic))
    f.close()

In [38]:
vocabulary = {}
inverted_index = {}
indexOfWord = 0
for doc in docpaths:
    with open('doc\{}'.format(doc), 'r',encoding="utf8") as file:
        lines = file.readlines()
        if len(lines) > 0 :
            document = lines[0]
            cols = document.replace('\\n',' ').split('\t')[:-1]
            document = '\t'.join(cols)
            tokens = nltk.word_tokenize(document)
            filtered_words = [word.lower() for word in tokens if word not in nltk.corpus.stopwords.words('english') and word not in ".,'()$"]
            for w in filtered_words:
                if w not in vocabulary:
                    indexOfWord += 1
                    vocabulary[w] = indexOfWord
                if vocabulary[w] not in inverted_index:
                    temp = []
                    temp.append(doc)
                    inverted_index[vocabulary[w]] = temp
                elif doc not in inverted_index[vocabulary[w]]:
                    inverted_index[vocabulary[w]].append(doc)
save_dict_to_file(inverted_index,"inverted_index")
save_dict_to_file(vocabulary,"vocabulary")
        


In [39]:
print(vocabulary)

{'27': 1, '2': 2, 'humble': 3, 'may': 4, '2016': 5, 'welcome': 6, 'stay': 7, 'private': 8, 'room': 9, 'queen': 10, 'bed': 11, 'detached': 12, 'bathroom': 13, 'second': 14, 'floor': 15, 'another': 16, 'bedroom': 17, 'sofa': 18, 'available': 19, 'additional': 20, 'guests': 21, '10': 22, 'guest': 23, '10min': 24, 'iah': 25, 'airport': 26, 'pick-up/drop': 27, '10/trip': 28, '30.0201379199512': 29, '-95.2939960042513': 30, 'rooms/bathroom': 31, '149': 32, '4': 33, 'san': 34, 'antonio': 35, 'november': 36, '2010': 37, 'stylish': 38, 'fully': 39, 'remodeled': 40, 'home': 41, 'upscale': 42, 'nw': 43, '–': 44, 'alamo': 45, 'heights': 46, 'area': 47, 'amazing': 48, 'location': 49, '-': 50, 'house': 51, 'conveniently': 52, 'located': 53, 'quiet': 54, 'street': 55, 'beautiful': 56, 'seasoned': 57, 'trees': 58, 'prestigious': 59, 'neighborhood': 60, 'close': 61, '281': 62, '410': 63, 'loop': 64, 'down-town': 65, 'featuring': 66, 'open': 67, 'plan': 68, 'original': 69, 'hardwood': 70, 'floors': 71, 

In [40]:
print(inverted_index)

{1: ['doc_1.tsv'], 2: ['doc_1.tsv', 'doc_2.tsv', 'doc_5.tsv'], 3: ['doc_1.tsv'], 4: ['doc_1.tsv'], 5: ['doc_1.tsv', 'doc_4.tsv', 'doc_6.tsv', 'doc_7.tsv', 'doc_8.tsv', 'doc_9.tsv'], 6: ['doc_1.tsv', 'doc_5.tsv'], 7: ['doc_1.tsv'], 8: ['doc_1.tsv', 'doc_4.tsv', 'doc_8.tsv'], 9: ['doc_1.tsv', 'doc_2.tsv', 'doc_3.tsv', 'doc_4.tsv', 'doc_8.tsv'], 10: ['doc_1.tsv', 'doc_8.tsv'], 11: ['doc_1.tsv', 'doc_8.tsv'], 12: ['doc_1.tsv'], 13: ['doc_1.tsv', 'doc_4.tsv', 'doc_8.tsv'], 14: ['doc_1.tsv'], 15: ['doc_1.tsv', 'doc_2.tsv'], 16: ['doc_1.tsv'], 17: ['doc_1.tsv', 'doc_4.tsv', 'doc_8.tsv', 'doc_9.tsv'], 18: ['doc_1.tsv'], 19: ['doc_1.tsv', 'doc_8.tsv'], 20: ['doc_1.tsv'], 21: ['doc_1.tsv'], 22: ['doc_1.tsv'], 23: ['doc_1.tsv'], 24: ['doc_1.tsv'], 25: ['doc_1.tsv'], 26: ['doc_1.tsv', 'doc_2.tsv', 'doc_7.tsv'], 27: ['doc_1.tsv'], 28: ['doc_1.tsv'], 29: ['doc_1.tsv'], 30: ['doc_1.tsv'], 31: ['doc_1.tsv'], 32: ['doc_2.tsv'], 33: ['doc_2.tsv', 'doc_6.tsv'], 34: ['doc_2.tsv', 'doc_3.tsv'], 35: ['doc_2

In [ ]:
query = input()